<a href="https://colab.research.google.com/github/ArturHugo/PLN-2022-1/blob/main/Atividade05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Atividade 05 - Competição

In [151]:
import numpy as np
import pandas as pd
import nltk
import collections
import math

In [152]:
df = pd.read_csv('https://raw.githubusercontent.com/viniciusrpb/cic0269_natural_language_processing/main/datasets/corpora/training.csv')
df = df.sample(frac=1)
df

,Id,text,Expected
1067,1067,Hotel muito bem localizado próximo ao shopping...,2
3067,3067,Possui três torres com variações nas acomodaçõ...,2
3369,3369,Com uma localização estratégica no centro de B...,2
1924,1924,Fiquei diversos dias lá e vou de novo. Apto co...,2
3605,3605,"Muito bom. Apenas , faço uma ressalva em relaç...",2
...,...,...,...
3404,3404,"Muito bem localizado, ótimo café da manhã e ót...",2
2791,2791,"As suites são muito amplas, tem boa disponibil...",2
477,477,"Quartos limpos e silenciosos, atendimento soli...",1
2420,2420,"O hotel está bem cuidado, tudo aparentemente n...",1


In [153]:
textos = df.text.to_list()

## Tagging de classes gramaticais


## Classificação de polaridade usando dicionário

### Representação TFIDF

In [154]:
# from nltk.corpus import stopwords
# stop_list = stopwords.words('portuguese')
# stemmer = nltk.stem.porter.PorterStemmer()
# nltk.download('stopwords')
# def preproc(text):
#   words = text.lower().split()
#   words = [word for word in words if word not in stop_list]
#   words = [stemmer.stem(word) for word in words]
#   return words

# preproc("A melhor vida que o hotel batatinha EspArrama")

#### Calculando Bag of Words

In [155]:
# bow_dict = {}
# for i,doc in enumerate(textos):
#     doc_name = df.Id[i]
#     bow_dict[doc_name] = dict()
#     for word in preproc(doc):
#         bow_dict[doc_name][word] = 1 if word not in bow_dict[doc_name] else bow_dict[doc_name][word]+1
        
# bow = pd.DataFrame().from_records(bow_dict).fillna(0).T.astype(int)

In [156]:
# tf = {}

# for i, doc in enumerate(textos):

#     doc_name = df.Id[i]
#     tf[doc_name] = dict()
#     words = preproc(doc)

#     for word in words:
#         tf[doc_name][word] = (1 if  word not in tf[doc_name] 
#                                     else tf[doc_name][word]+1)
#     tf[doc_name] = {word:(freq/len(words)) 
#                     for word, freq in tf[doc_name].items()}

# idf =  {}

# vocab = bow.columns
# idf =  {}

# for word in vocab:
#     idf[word] = math.log(len(bow)/len(bow[word][bow[word]!=0]))

# tfidf_dict = {}

# for doc in tf:
#     tfidf_dict[doc] = dict()
#     for word in tf[doc]:
#         tfidf_dict[doc][word] = tf[doc][word]*idf[word]

# tfidf = pd.DataFrame().from_records(tfidf_dict).fillna(0).T

In [157]:
# tfidf

## TFIDF DE VERDADE

In [158]:
 from sklearn.feature_extraction.text import TfidfVectorizer

In [159]:
training = textos[0:1000]
validation = textos[-10:]

In [160]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(training).toarray()

In [161]:
X_test = vectorizer.transform(validation).toarray()

### KNN

In [162]:
# Cosine similarity
def cosine_similarity(v1, v2):
  return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [163]:
# Classify sample based on K neares neighbors
def knn_classify(sample_array, dataset_array, dataset_df, K, similarity_function):
  indexes = np.arange(dataset_array.shape[0])
  similarities = []
  for vector in dataset_array:
    similarities.append(similarity_function(sample_array, vector))

  index_similarities = list(zip(similarities, indexes))
  sorted_index_similariteies = sorted(index_similarities, reverse=True)
  nearest_neighbors = [ s for _, s in sorted_index_similariteies[:K]]
  count_labels = collections.Counter(dataset_df['Expected'][nearest_neighbors])
  return max(count_labels, key=count_labels.get)

In [164]:
# training = tfidf.values[0:3500]
# validation = tfidf.values[-3]

In [165]:
predictions = [knn_classify(sample, X, df, 10, cosine_similarity) for sample in X_test]

In [166]:
preds = list(zip(predictions, df['Expected'].values[1000:]))
accuracy = [(x==y) for x, y in preds]
np.sum(accuracy)/len(accuracy)




0.7